In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import json 

from pathlib import Path

import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch import optim


from ml_utils import CustomDataset
from training import train_classifier
from model.resnet_model import CNNClassifier

In [ ]:
train_dt = CustomDataset(data_path=Path("../datasets/car_dataset"), portion="train")
train_loader = DataLoader(train_dt,
                          batch_size=150, shuffle=True, num_workers=4)

valid_dt = CustomDataset(data_path=Path("../datasets/car_dataset"), portion="valid")
valid_loader = DataLoader(valid_dt,
                          batch_size=150, shuffle=True, num_workers=4)

In [ ]:
cls = CNNClassifier(num_classes=196, pretrained=True)
# cls.load_state_dict(torch.load("./models/resnet_finetune_50e.pth", weights_only=True))
# cls.freeze_first_n_layers(n_layers=7)

In [ ]:
metric_history = Path("./saved_models/test1/test.json")
with metric_history.open("w") as file:
    json.dump([], file)

In [ ]:
# !export CUDA_VISIBLE_DEVICES=3

In [ ]:
train_classifier(
    model=cls,
    dataset=(train_loader, valid_loader),
    optimizer=optim.Adam,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    num_epochs=1,
    opt_params=dict(lr=0.001),
    sc_params=dict(mode="max", patience=3, threshold=0.9),
    metric_history=metric_history,
    model_cache="./saved_models/test1/model.pth",
    patience=5,
    device='cuda:0',
    current_state=dict()
)

In [ ]:
# hist_dt = pd.read_json(metric_history)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_style("darkgrid")

In [ ]:
# f, axs = plt.subplots(nrows=1, ncols=2, figsize = (16,5))
#
# epochs = [*range(len(hist_dt))]
#
#
# sns.lineplot(x=epochs, y=hist_dt.train_loss, ax=axs[0], label='Training Loss')
# axs[0].fill_between(epochs, hist_dt.train_loss - hist_dt.train_loss_std, hist_dt.train_loss + hist_dt.train_loss_std, alpha=0.2)
#
#
# sns.lineplot(x=epochs, y=hist_dt.valid_loss, ax=axs[0], label='Validation Loss')
# axs[0].fill_between(epochs, hist_dt.valid_loss - hist_dt.valid_loss_std, hist_dt.valid_loss + hist_dt.valid_loss_std, alpha=0.2)
#
# sns.lineplot(x=epochs,y=hist_dt.valid_f1, ax=axs[1], label='Validation F1 Score')
# sns.lineplot(x=epochs,y=hist_dt.valid_acc, ax=axs[1], label='Validation Accuracy')
#
# axs[0].set_title('Training and Validation Loss')
# axs[0].set_xlabel('Epoch', fontsize=12)
# axs[0].set_ylabel('Loss', fontsize=12)
# axs[0].set_xticks(epochs)
# axs[0].set_xticklabels([i if i%1==0  else "" for i in epochs])
# #axs[0].set_ylim(0,5)
# axs[0].grid(True)
#
# axs[1].set_title('F1 validation')
# axs[1].set_xlabel('Epoch', fontsize=12)
# axs[1].set_ylabel('F1', fontsize=12)
# axs[1].set_xticks(epochs)
# axs[1].set_xticklabels([i if i%1==0  else "" for i in epochs])
# axs[1].grid(True)
#
# plt.tight_layout()
# plt.show()